# 🧠 Chatbot RAG con Memoria para E-commerce  – por Mauricio Rostagno

Este proyecto es un prototipo funcional de **chatbot inteligente** desarrollado como parte de mi portafolio de aplicaciones con inteligencia artificial y lenguaje natural. Uso a modo de simulación la necesidad de asistir a clientas de un emprendimiento online, pero puede adaptarse fácilmente a cualquier comercio que quiera ofrecer atención automatizada por redes o web.

---

## ⚙️ Tecnologías utilizadas

- **LangChain**: Framework modular para construir agentes y pipelines con LLMs.
- **ChromaDB**: Vector store local para almacenar embeddings y realizar búsquedas semánticas.
- **HuggingFace Transformers**: Para usar modelos como `flan-t5-large` (entrenado para tareas de texto).
- **Groq + LLaMA3**: Modelo de lenguaje potente, rápido y gratuito para generar respuestas naturales.
- **Python + Google Colab**: Desarrollo y ejecución del prototipo.

---

## 🧩 ¿Qué hace este chatbot?

- Responde preguntas sobre el negocio, productos y formas de pago.
- Recupera contexto relevante con un pipeline **RAG (Retrieval-Augmented Generation)**.
- Usa **memoria conversacional** para mantener coherencia en el diálogo.


---

## 💡 Próximos pasos en los que trabajo

- Integración con Google Sheets para leer productos actualizados.
- Montaje de una interfaz visual (ej. Streamlit o Gradio).
- Posible despliegue como asistente en redes sociales o WhatsApp Business.

---

📬 Contacto: rostagno.mj@gmail.com  
🔗 GitHub: [@MauriRos](https://github.com/MauriRos)  
🌐 LinkedIn: [linkedin.com/in/mauricio-rostagno](https://www.linkedin.com/in/mauricio-rostagno)


## 🔧 Paso 0: Instalación de librerías necesarias

En esta celda instalamos todas las dependencias para construir nuestro chatbot RAG con memoria.

Incluye:
- `LangChain`: framework para aplicaciones con LLMs.
- `ChromaDB` y `FAISS`: almacenamiento vectorial.
- `Transformers`: modelos de lenguaje de Hugging Face.
- `sentence-transformers`: modelos de embeddings.
- `langchain-groq`: para conectarnos con el modelo LLaMA 3 desde la API de Groq.


In [ ]:
# Instalamos las librerías necesarias
!pip install langchain langchain-community langchain-core
!pip install openai chromadb faiss-cpu tiktoken
!pip install sentence-transformers
!pip install transformers
!pip install langchain-groq


## 📚 Paso 1: Importación de librerías

En este bloque importamos todas las librerías que usaremos en el proyecto, tanto para manejo de datos como para crear la lógica del chatbot.

Incluye:
- Librerías de Python (`numpy`, `pandas`)
- Componentes clave de `LangChain`
- El pipeline de modelos de `transformers`


In [ ]:
# Librerías básicas
import numpy as np
import pandas as pd

# LangChain
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Transformers
from transformers import pipeline


## 🛍️ Paso 2: Definir la información del negocio

Aquí cargamos los textos que contienen la información real de ViaéBags Marroquinería: qué productos vende, cómo se realizan los pagos y envíos, formas de contacto, etc.

Estos textos serán utilizados como base de conocimiento del asistente virtual.


In [ ]:
sample_texts = [
   "ViaéBags Marroquinería es un emprendimiento de Santa Fe que vende carteras, bolsos, mochilas y equipaje por Instagram.",
    "No contamos con showroom ni local físico. Solo ofrecemos retiro por domicilio particular o envío a todo el país.",
    "Realizamos envíos a cualquier parte de Argentina mediante servicios de correo o retiro por domicilio. No hay atención presencial.",
    "Podés retirar por un punto de la ciudad de Santa Fe si estás cerca. Coordinamos el punto exacto por mensaje.",
    "Aceptamos todos los medios de pago: tarjetas de crédito, débito, transferencias y Mercado Pago.",
    "Consultanos por descuentos en efectivo o promociones especiales vigentes.",
    "Vendemos varias marcas reconocidas y destacamos por tener los mejores precios del mercado.",
    "Frecuentemente realizamos sorteos y promociones en nuestra cuenta de Instagram: @viae.bags.",
    "Atendemos consultas por Instagram y por correo electrónico: viabagsonline@gmail.com.",
    "Respondemos con amabilidad, claridad y cercanía todas las dudas sobre productos, precios, medios de pago y disponibilidad.",
    "Podés consultarnos por catálogo de productos, disponibilidad de stock o promociones activas."
]


## 📄 Paso 3: Convertir textos en objetos `Document`

Los textos cargados se convierten en objetos `Document`, un formato compatible con LangChain que permite luego procesarlos, dividirlos y vectorizarlos para el sistema de recuperación de información.

Esto nos permite que cada fragmento tenga estructura y pueda usarse dentro del flujo RAG.


In [ ]:
from langchain.schema import Document

# Convertimos los textos en objetos Document
documents = [Document(page_content=text) for text in sample_texts]


## ✂️ Paso 4: Dividir textos en fragmentos (chunks)

Dividimos cada documento en fragmentos de máximo 200 caracteres, con un solapamiento de 50. Esto mejora la recuperación posterior, ya que permite encontrar secciones más relevantes dentro del contexto original.

LangChain se encarga automáticamente de cortar y preservar la coherencia del texto.


In [ ]:
from langchain.text_splitter import CharacterTextSplitter

# Creamos el splitter (divisor de texto)
text_splitter = CharacterTextSplitter(
    chunk_size=200,        # Máximo 200 caracteres por fragmento
    chunk_overlap=50       # Solapamiento de 50 caracteres entre fragmentos
)

# Aplicamos el splitter a nuestros documentos
chunked_documents = text_splitter.split_documents(documents)


## 🧠 Paso 5: Generar embeddings y crear la base vectorial

En este paso transformamos los textos en vectores numéricos (embeddings) utilizando el modelo `all-MiniLM-L6-v2` de Hugging Face. Luego almacenamos esos vectores en una base vectorial llamada **ChromaDB**, que nos permitirá buscar información relevante a partir de una consulta.

📦 Esta base queda guardada en el disco (`./chroma_db`) para reutilizarla más adelante.


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Creamos el modelo de embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
persist_dir = "./chroma_db"
# Creamos la base vectorial con ChromaDB
chroma_db = Chroma.from_documents(documents=chunked_documents, embedding=embedding_model, persist_directory=persist_dir)

chroma_db.persist()


## ⚡ Paso 6: Conectar el modelo LLaMA 3 vía Groq

En esta celda conectamos nuestro chatbot a la API de **Groq**, que nos permite usar el modelo **LLaMA 3** de forma gratuita y extremadamente rápida.

El modelo se invoca a través del componente `ChatGroq`, que se integra fácilmente con LangChain. Esta es la inteligencia central que va a generar las respuestas.


In [ ]:
# 📦 Requiere instalar groq primero (ya lo hiciste, si no: !pip install groq)
from langchain_groq import ChatGroq
import os

# Pegá tu Groq API key
os.environ["GROQ_API_KEY"] = "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"  # ← reemplazá esto por tu key real

# Crear el LLM de Groq usando LLaMA3
llm = ChatGroq(
    groq_api_key=os.environ["GROQ_API_KEY"],
    model_name="llama3-8b-8192"
)


## 🧩 Paso 7: Crear el prompt, el retriever y el pipeline RAG

En este bloque definimos tres componentes clave:

- **Prompt personalizado:** Le da instrucciones al modelo sobre cómo debe responder, incluyendo el tono y el contexto.
- **Retriever:** Recupera fragmentos relevantes desde la base Chroma a partir de la consulta del usuario.
- **Pipeline RAG (Retrieval-Augmented Generation):** Une todo en una cadena secuencial: recibe una pregunta, recupera contexto, construye el prompt y genera una respuesta.

Además, agregamos un paso intermedio para **mostrar el prompt por consola**, ideal para debugging y entender qué está viendo el modelo.


In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableMap
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import RunnableLambda

# 1. Crear un prompt template (como antes)
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Sos un asistente virtual de ViaéBags Marroquinería, una tienda de carteras y bolsos.
Respondé con claridad y solo usando esta información:

{context}

Pregunta: {question}

Respuesta:
"""
)

# Esta función une el contexto y la pregunta en un string prompt listo para el modelo
build_prompt = RunnableLambda(lambda x: prompt_template.format(**x))

# 2. Crear el retriever
retriever = chroma_db.as_retriever(search_type="similarity", search_kwargs={"k": 2})


# Función que extrae la pregunta del input completo
#extract_question = RunnableLambda(lambda x: x["question"])

# Procesar el contexto para convertir documentos a texto plano
def extract_context(docs):
    if isinstance(docs, list):
        return "\n".join([doc.page_content for doc in docs])
    return docs

extract_context_lambda = RunnableLambda(extract_context)

from langchain_core.runnables import RunnableLambda

# Esto te muestra por pantalla el prompt que se le envía al modelo
debug_prompt = RunnableLambda(lambda x: print("📤 Prompt al modelo:\n", x) or x)

# Creamos la nueva cadena con filtro
rag_chain = (
    {
        "context": extract_question | retriever | extract_context_lambda,
        "question": extract_question
    }
    | build_prompt
    #| debug_prompt
    | llm
    | StrOutputParser()
)


## 🧠 Paso 8: Incorporar memoria conversacional

Este paso nos permite que el chatbot recuerde los mensajes anteriores de cada usuario. Así se logra una conversación más coherente, donde el modelo puede responder de forma contextual.

🔁 Usamos `RunnableWithMessageHistory` para conectar el pipeline RAG con un historial de chat en memoria. Esto asocia cada sesión de usuario (por ejemplo: `"cliente123"`) con su conversación previa.

Finalmente, hacemos una primera prueba enviando la pregunta: `"¿Puedo pagar con tarjeta?"`.


In [ ]:
from langchain.memory.chat_message_histories.in_memory import ChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory.chat_message_histories.in_memory import ChatMessageHistory

def get_message_history(session_id: str):
    return ChatMessageHistory()
rag_with_memory = RunnableWithMessageHistory(
    rag_chain,
    get_message_history,
    input_messages_key="question",
    history_messages_key="history"
)
response = rag_with_memory.invoke(
    {"question": "¿Puedo pagar con tarjeta?"},
    config={"configurable": {"session_id": "cliente123"}}
)
print("🤖", response)



🤖 ¡Claro! En ViaéBags Marroquinería, aceptamos pagos con tarjeta. Podrás elegir entre varias opciones de pago seguras y convenientes para ti. ¡No hay mejor manera de disfrutar de nuestros productos reconocidos y de nuestras ofertas exclusivas que con la comodidad de pagar con tarjeta!


In [ ]:
response = rag_with_memory.invoke(
    {"question": "¿Qué productos venden?"},
    config={"configurable": {"session_id": "cliente1"}}
)
print("🤖", response)

response = rag_with_memory.invoke(
    {"question": "¿Puedo pagar con tarjeta?"},
    config={"configurable": {"session_id": "cliente1"}}
)
print("🤖", response)

response = rag_with_memory.invoke(
    {"question": "¿Y si quiero devolverlo?"},
    config={"configurable": {"session_id": "cliente1"}}
)
print("🤖", response)

response = rag_with_memory.invoke(
    {"question": "¿Dónde se puede retirar?"},
    config={"configurable": {"session_id": "cliente1"}}
)
print("🤖", response)




🤖 ¡Hola! En ViaéBags Marroquinería, vendemos una amplia variedad de productos relacionados con carteras y bolsos. Ofrecemos una selección de modelos de carteras para hombres y mujeres, desde pequeñas bolsas de mano hasta grandes bolsos de viaje. También tenemos una gran variedad de accesorios, como cinturones, corbata y otros complementos para completar tu look. ¡Si necesitas algo en particular, no dudes en preguntar!
🤖 ¡Claro! En ViaéBags Marroquinería, aceptamos pagos con tarjeta de crédito o débito, ofreciendo la mayor comodidad y seguridad para tus compras.
🤖 Excelente elección! En ViaéBags Marroquinería, nos esforzamos por ofrecer productos de alta calidad y precios competitivos. Si deseas devolver un producto, no hay problema. Puedes hacerlo dentro de un plazo de [insertar plazo de devolución] después de la fecha de compra, siempre y cuando el producto esté en perfecto estado y tenga todos los elementos originales. Debes presentar la factura y el producto en nuestra tienda para i